<a href="https://colab.research.google.com/github/JacobChunn/trading-transformer/blob/time_series_optimizations/trading-transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trading Transformer

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    USING_COLAB = True
except ModuleNotFoundError:
    print("Google Colab environment not detected, running locally.")

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.cuda.amp import GradScaler, autocast
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np
import pandas as pd

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, bias):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        # Generate Scaled Gaussian Prior Bias & Time Splitter Mask (Ding et al., 2021) only at construction
        self.gaussian_prior = self.generate_gaussian_prior(50, bias)
        #self.timescale_mask = self.generate_timescale_mask(50, bias)
        #torch.set_printoptions(profile="full")
        #print("timescale_mask: ", self.timescale_mask)
        #torch.set_printoptions(profile="default")

    # Generate a mask with square sub matrices of a given side on the diagonal
    def generate_timescale_mask(self, size, sub_size):
        print("sub_size: ", sub_size)
        t = torch.full((size, size), -1e4)
        offset = 0
        for i in range(size):
            if (i+1) % (sub_size+1) == 0:
                offset = offset + sub_size
            for j in range(size):
                if j < offset:
                    continue
                elif (j-offset) > (sub_size-1):
                    break
                else:
                    t[i, j] = 0
        return t
    
    def generate_gaussian_prior(self, size, scale):
        g = torch.full((size, size), 0)
        for i in range(size):
            for j in range(size):
                if j > i:
                    break
                else:
                    g[i, j] = math.exp(-(math.pow((j - i), 2)/2*math.pow(scale, 2)))
        return g

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        attn_scores = attn_scores + self.gaussian_prior # Apply Gaussian Prior
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e4) # changed from -1e9
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [ ]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

## Embedding Class

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout, bias):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, bias)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

## Base Transformer Model

In [ ]:
class NextValuePredictor(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout, bias):
        super(NextValuePredictor, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout, bias*(_+1)) for _ in range(num_layers)])
        self.fc = nn.Linear(d_model, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, tgt):
        #print("tgt", tgt)
        device = tgt.device
        seq_length = tgt.size(1)
        #print("seq_length", seq_length)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(device)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(2) & nopeak_mask
        #print("tgt_mask: ", tgt_mask)
        #print("tgt_mask.shape: ", tgt_mask.shape)
        return tgt_mask

    def forward(self, tgt):
        tgt_mask = self.generate_mask(tgt)
        tgt_embedded = self.dropout(self.positional_encoding(self.embedding(tgt)))
        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, tgt_mask)
        output = self.fc(dec_output)
        return output

## Data Preprocessing

In [ ]:
# Read and preprocess the data
if USING_COLAB:
    data = pd.read_csv('/content/drive/Trading/Data/EURUSD-short.txt', skiprows=1, header=None)
else:
    data = pd.read_csv('EURUSD-short.txt', skiprows=1, header=None)
data.columns = ['<TICKER>', '<DTYYYYMMDD>', '<TIME>', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>']
data.columns = [col.strip('<>') for col in data.columns]

data['TIME'] = data['TIME'].apply(lambda x: f'{int(x):06d}')
data['DATETIME'] = pd.to_datetime(data['DTYYYYMMDD'].astype(str) + data['TIME'], format='%Y%m%d%H%M%S')

# Extract relevant columns
data = data[['DATETIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']]

In [ ]:
# Create sequences using sliding window of window_size
def create_full_sequences(data, window_size, predict_interval_count):
    sequences = []
    for i in range(0, len(data) - window_size, predict_interval_count):
            sequences.append(data.iloc[i:i + window_size].values)
    return np.array(sequences)

predict_interval_count = 1 # Num of values to predict for each sequence
window_size_indep = 50 # Num of intervals that spans across an input or target data val independently
window_size = window_size_indep + predict_interval_count  # Num of intervals that spans across each input and target data val combined
batch_size = 64    # Number of sequences per batch
vocab_size = 20000 # number of classes (unique tokens)
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 512  # orig val is 2048
max_seq_length = window_size  # Set to window_size - need to look into how this relates to window_size
dropout = 0.1

sequences = create_full_sequences(data["OPEN"], window_size, predict_interval_count)
#batch_sequences = get_batch(sequences, batch_size)

# Convert to PyTorch tensor
data_tensor = torch.tensor(sequences, dtype=torch.float32)

data_tensor_int = (data_tensor * 10000).long()

# Define the train-validation split
train_input = data_tensor_int[:, :-1]
train_target = data_tensor_int[:, 1:]

# Create a TensorDataset
dataset = TensorDataset(train_input, train_target)

# Define the train-validation split
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
# Base Time-Scale Biasing (5-minutes)
bias = 5
# Model instance
model = NextValuePredictor(vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout, bias).to(device)

# Training
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss() #using cross entropy instead
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

# Initialize GradScaler
scaler = GradScaler()

In [ ]:
model.train()

for epoch in range(25):
    # Training phase
    for batch_input, batch_target in train_loader:
        # Move data to the GPU
        batch_input, batch_target = batch_input.to(device), batch_target.to(device)
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Autocast context manager
        with autocast():
            # Forward pass with mixed precision
            output = model(batch_input)
            # Compute loss with mixed precision
            loss = criterion(output.contiguous().view(-1, vocab_size), batch_target.contiguous().view(-1))
        # Backward pass with scaled loss to prevent underflow
        scaler.scale(loss).backward()
        # Update the parameters
        scaler.step(optimizer)
        # Update the scale for next iteration
        scaler.update()
        print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

    # Validation phase
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_input, batch_target in val_loader:
            batch_input, batch_target = batch_input.to(device), batch_target.to(device)
            output = model(batch_input)
            loss = criterion(output.view(-1, vocab_size), batch_target.view(-1))
            val_loss += loss.item()

    # Average validation loss
    val_loss /= len(val_loader)
    print(f"Epoch {epoch + 1}, Training Loss: {loss.item()}, Validation Loss: {val_loss}")
    
    # Set the model back to training mode
    model.train()

In [ ]:
# Final evaluation phase
model.eval()
test_loss = 0
with torch.no_grad():
    for batch_input, batch_target in test_loader:
        batch_input, batch_target = batch_input.to(device), batch_target.to(device)
        output = model(batch_input)
        loss = criterion(output.view(-1, vocab_size), batch_target.view(-1))
        test_loss += loss.item()

# Average test loss
test_loss /= len(test_loader)
print(f"Test Loss: {test_loss}")